In [3]:
import pandas as pd
import requests
from tqdm import tqdm_notebook

def get_doi_from_issn(issn, rows=1000):
    '''
    Function for uploading DOI from api.crossref.org
    input: ISNN: [str]
    return: DOI: [list]
    '''
    all_dois = {}
    cursor = '*'
    d = requests.get(f'https://api.crossref.org/v1/journals/{issn}/works?sort=published&rows={str(rows)}&cursor={cursor}').json()
    all_dois.update({i['DOI']:{'title': i['title'][0], 
                               'year': i['published']['date-parts'][0][0]} for i in d['message']['items']})
    total_results = d['message']['total-results']
    next_cursor = d['message']['next-cursor']

    for i in tqdm_notebook(range(int(total_results/rows))):
        d = requests.get(f'https://api.crossref.org/v1/journals/{issn}/works?sort=published&rows=1000&cursor={next_cursor}').json()
        next_cursor = d['message']['next-cursor']
        try:
            all_dois.update({i['DOI']:{'title': i['title'][0], 
                                       'year': i['published']['date-parts'][0][0]} for i in d['message']['items']})
        except:
            pass
    return all_dois

In [4]:
#For example, The Journal of Chemical Thermodynamics
issn = '0021-9614'
dois = get_doi_from_issn(issn)

<ipython-input-3-b1b37f670232>:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(int(total_results/rows))):


  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
solub = {doi:item for doi, item in dois.items() if 'solub' in item['title'].lower()}

In [6]:
solub_df = pd.DataFrame({'DOI': list(solub.keys()), 
                         'Title': [item['title'] for item in list(solub.values())], 
                         'Year': [item['year'] for item in list(solub.values())]})